In [1]:
from bs4 import BeautifulSoup
import operator
import time
from selenium import webdriver
import os
from collections import defaultdict, Counter
import pandas as pd
import numpy as np

from common.styling import *

In [2]:
def get_week_stats(scoreboard_html_source, league='NHL'):
    matchup_results = []
    matchups = scoreboard_html_source.findAll('div', {'Scoreboard__Row'})
    for matchup in matchups:
        opponents = matchup.findAll('li', 'ScoreboardScoreCell__Item')
        res = []
        for opp in opponents:
            team = opp.findAll('div', {'class': 'ScoreCell__TeamName'})[0].text
            if league == 'NHL':
                score = float(opp.findAll('div', {'class': 'ScoreCell__Score'})[0].text)
            else:
                score_nba = opp.findAll('div', {'class': 'ScoreCell__Score'})[0].text.split('-')
                score = '-'.join(map(lambda x: str(x) if x % 1.0 > 1e-7 else str(int(x)), map(float, score_nba)))
            res.append((team, score))
        matchup_results.append(res)
    return matchup_results


def get_espn_fantasy_hockey_scoreboard_stats(league_id, n_weeks):
    espn_scoreboard_url = 'https://fantasy.espn.com/hockey/league/scoreboard'
    urls = [f'{espn_scoreboard_url}?leagueId={league_id}&matchupPeriodId={i+1}' for i in range(n_weeks)]
    all_matchups = []
    browser = webdriver.Chrome()
    for item in urls:
        browser.get(item)
        time.sleep(8)
        html_soup = BeautifulSoup(browser.page_source)
        all_matchups.append(get_week_stats(html_soup))
    return all_matchups


def get_sorted_opponent_week_scores(week_matchups):
    opponent_scores = []
    for matchup in week_matchups:
        opponent_scores.append((matchup[0][0], matchup[1][1]))
        opponent_scores.append((matchup[1][0], matchup[0][1]))
    return sorted(opponent_scores, key=operator.itemgetter(1), reverse=True)

def get_sorted_week_scores(week_matchups):
    scores = []
    for matchup in week_matchups:
        scores.extend(matchup)
    return sorted(scores, key=operator.itemgetter(1), reverse=True)


def get_places(sorted_scores):
    places = {}
    i = 1
    while i <= len(sorted_scores):
        j = i + 1
        while j <= len(sorted_scores) and sorted_scores[j-1][1] == sorted_scores[i-1][1]:
            j += 1
        place = (i + j - 1) / 2
        for k in range(i, j):
            places[sorted_scores[k-1][0]] = place
        i = j
    return places


def get_lucky_score(matchups, places):
    lk = {}
    for player1, player2 in matchups:
        if player1[1] > player2[1]:
            place1 = places[player1[0]]
            lk[player1[0]] = max(0, place1 - len(places) / 2)

            place2 = places[player2[0]]
            lk[player2[0]] = min(0, place2 - len(places) / 2 - 1)
        elif player1[1] < player2[1]:
            place1 = places[player1[0]]
            lk[player1[0]] = min(0, place1 - len(places) / 2 - 1)

            place2 = places[player2[0]]
            lk[player2[0]] = max(0, place2 - len(places) / 2)
        else:
            place = places[player1[0]]
            if place > len(places) / 2:
                lk[player1[0]] = (place - len(places) / 2) / 2
                lk[player2[0]] = (place - len(places) / 2) / 2
            else:
                lk[player1[0]] = (place - len(places) / 2 - 1) / 2
                lk[player2[0]] = (place - len(places) / 2 - 1) / 2
    return lk


def get_stats(leagues, n_weeks):
    last_week_scores = []
    all_weeks_scores = []
    for league in leagues:
        
        espn_scoreboard_url = 'https://fantasy.espn.com/hockey/league/scoreboard'
        urls = [f"{espn_scoreboard_url}?leagueId={league}&matchupPeriodId={i+1}" for i in range(n_weeks)]
        browser = webdriver.Chrome()

        result_lk = defaultdict(list)
        opponent_lk = defaultdict(list)
        result_places = defaultdict(list)
        opponent_places = defaultdict(list)
        for index, url in enumerate(urls):
            browser.get(url)
            time.sleep(8)
            content = browser.page_source
            data = BeautifulSoup(content)
            results = get_week_stats(data)
            
            opp_dict = {}
            for sc in results:
                opp_dict[sc[0][0]] = sc[1][0]
                opp_dict[sc[1][0]] = sc[0][0]
            
            scores = get_sorted_week_scores(results)
            opp_scores = get_sorted_opponent_week_scores(results)
            
            league_name = get_league_name(data)
            for score in scores:
                all_weeks_scores.append((score[0], score[1], index+1, league_name))
                if index == n_weeks - 1:
                    last_week_scores.append((score[0], score[1], league_name))
                    
            places = get_places(scores)

            lk = get_lucky_score(results, places)
            for item in lk:
                result_places[item].append(places[item] if places[item] % 1.0 != 0 else int(places[item]))
                result_lk[item].append(lk[item] if lk[item] % 1.0 > 1e-7 else int(lk[item]))
            
            for team in opp_dict:
                opp_place = places[opp_dict[team]]
                opponent_places[team].append(opp_place if opp_place % 1.0 != 0 else int(opp_place))
                opp_lk = lk[opp_dict[team]]
                opponent_lk[team].append(opp_lk if opp_lk % 1.0 != 0 else int(opp_lk))

        for key in result_lk:
            result_lk[key].append(np.sum(result_lk[key]))
        for key in result_places:
            result_places[key].append(np.sum(result_places[key]))
        for key in opponent_lk:
            opponent_lk[key].append(np.sum(opponent_lk[key]))
        for key in opponent_places:
            opponent_places[key].append(np.sum(opponent_places[key]))
    
        weeks = [f'Week {i+1}' for i in range(n_weeks)]
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        
        df_lk = pd.DataFrame(data=list(result_lk.values()),
                          index=result_lk.keys(), 
                          columns=weeks + ['SUM'])
        df_lk = df_lk.sort_values(['SUM'])
        display(df_lk.style.set_table_styles(styles).\
                applymap(color_value, subset=weeks).\
                set_caption(f'{league_name}: luck scores'))
        
        df_places = pd.DataFrame(data=list(result_places.values()),
                          index=result_places.keys(), 
                          columns=weeks + ['SUM'])
        df_places = df_places.sort_values(['SUM'])
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        display(df_places.style.set_table_styles(styles).\
                apply(color_place_column, subset=weeks).\
                set_caption(f'{league_name}: places'))                    
        
        df_opp_lk = pd.DataFrame(data=list(opponent_lk.values()),
                                 index=opponent_lk.keys(), 
                                 columns=weeks + ['SUM'])
        df_opp_lk = df_opp_lk.sort_values(['SUM'])
        display(df_opp_lk.style.set_table_styles(styles).\
                applymap(color_opponent_value, subset=weeks).\
                set_caption(f'{league_name}: opponent luck scores'))
        
        df_opp_places = pd.DataFrame(data=list(opponent_places.values()),
                          index=opponent_places.keys(), 
                          columns=weeks + ['SUM'])
        df_opp_places = df_opp_places.sort_values(['SUM'])
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        display(df_opp_places.style.set_table_styles(styles).\
                apply(color_opponent_place_column, subset=weeks).\
                set_caption(f'{league_name}: opponent places'))
                                         
    return df_lk, df_places, last_week_scores, all_weeks_scores


def get_league_name(scoreboard_html_source):
    return scoreboard_html_source.findAll('h3')[0].text

In [ ]:
leagues = [8290, 31769, 33730, 52338, 57256, 73362809]
a, b, c, d = get_stats(leagues, 11)

In [ ]:
week_scores_sorted = sorted(c, key=operator.itemgetter(1), reverse=True)

n_top = int(len(week_scores_sorted) / len(leagues))
data = pd.DataFrame(data = week_scores_sorted[:n_top], 
                    index=np.arange(1, 1 + n_top),
                    columns=['Team', 'Score', 'League'])
data

In [ ]:
all_scores_soted = sorted(d, key=operator.itemgetter(1), reverse=True)

data = pd.DataFrame(data = all_scores_soted[:20], 
                    index=np.arange(1, 1 + 20),
                    columns=['Team', 'Score', 'Week' ,'League'])
data

In [ ]:
leagues = [27465869]
a, b, c, d = get_stats(leagues, 1)

In [ ]:
week_scores_sorted = sorted(c, key=operator.itemgetter(1), reverse=True)

n_top = int(len(week_scores_sorted) / len(leagues))
data = pd.DataFrame(data = week_scores_sorted[:n_top], 
                    index=np.arange(1, 1 + n_top),
                    columns=['Team', 'Score', 'League'])
data

In [ ]:
all_scores_soted = sorted(d, key=operator.itemgetter(1), reverse=True)

data = pd.DataFrame(data = all_scores_soted[:20], 
                    index=np.arange(1, 1 + 20),
                    columns=['Team', 'Score', 'Week' ,'League'])
data

In [4]:
def get_week_results(scoreboard_html_source):
    matchups = scoreboard_html_source.findAll('div', {'Scoreboard__Row'})
    results = []
    for match in matchups:
        opponents = match.findAll('li', 'ScoreboardScoreCell__Item')
        team_names = []
        for opp in opponents:
            team_names.append(opp.findAll('div', {'class': 'ScoreCell__TeamName'})[0].text)

        rows = match.findAll('tr', {'Table2__tr'})
        categories = [header.text for header in rows[0].findAll('th', {'Table2__th'})[1:]]
        first_player_stats = [data.text for data in rows[1].findAll('td', {'Table2__td'})[1:]]
        second_player_stats = [data.text for data in rows[2].findAll('td', {'Table2__td'})[1:]]

        results.append(
            ((team_names[0], [(cat, float(stat)) for cat, stat in zip(categories, first_player_stats)]),
             (team_names[1], [(cat, float(stat)) for cat, stat in zip(categories, second_player_stats)])))
    return results, categories


def get_scores_info(results, categories):
    indexes = []
    data = []
    
    for matchup in results:
        for player, score in matchup:
            indexes.append(player)
            data.append([cat_score if cat_score % 1.0 > 1e-7 else int(cat_score) for cat, cat_score in score])
    return {indexes[i]: data[i] for i in range(len(indexes))}


def get_expected_category_stat(score_pairs, category):
    scores = np.array([score for team, score in score_pairs])
    result = {}
    for team, sc in score_pairs:
        win_count = np.sum(scores < sc)
        draw_count = np.sum(scores == sc) - 1
        lost_count = np.sum(scores > sc)
        if category == 'TO':
            result[team] = np.array([lost_count, win_count, draw_count]) / (len(scores) - 1)
        else:
            result[team] = np.array([win_count, lost_count, draw_count]) / (len(scores) - 1)
    return result


def compare_scores(sc1, sc2):
    return list(sc1[[0,2,1]]) > list(sc2[[0,2,1]])

def get_expected_score_and_result(results):
    res = {}
    indexes = []
    data = []
    for matchup in results:
        for player, score in matchup:
            indexes.append(player)
            res[player] = np.array([0.0, 0.0, 0.0])
            data.append([cat_score for cat, cat_score in score])
    opponents_dict = {}        
    for matchup in results:
        opponents_dict[matchup[0][0]] = matchup[1][0]
        opponents_dict[matchup[1][0]] = matchup[0][0]
    data = np.array(data)
    for i in range(len(data[0])):
        pairs = [(team, score) for team, score in zip(indexes, data[:, i])]
        category = results[0][0][1][i][0]
        expected_stats = get_expected_category_stat(pairs, category)
        for player in expected_stats:
            concatted = np.vstack((expected_stats[player], res[player]))
            res[player] = concatted.sum(axis = 0)
    matchup_results = {}
    for team in opponents_dict:
        if compare_scores(res[team], res[opponents_dict[team]]):
            matchup_results[team] = 'W'
        elif compare_scores(res[opponents_dict[team]], res[team]):
            matchup_results[team] = 'L'
        else:
            matchup_results[team] = 'D'
    exp_categories = {}
    for player in res:
        res_rounded = map(lambda x: np.round(x, 1), res[player])
        res_formatted = map(lambda x: str(x) if x % 1.0 > 1e-7 else str(int(x)), res_rounded)
        exp_categories[player] = '-'.join(res_formatted)
        
    return exp_categories, matchup_results
        

def get_places_table(vvv, categories):
    places_table_data = defaultdict(list)
    for col in list(vvv.columns):
        pairs = [(index, vvv[col][index]) for index in vvv[col].index]
        pairs_sorted = sorted(pairs, key=operator.itemgetter(1), reverse=True if col != 'TO' else False)
        places = get_places(pairs_sorted)
        for index in places:
            places_table_data[index].append(places[index])
    for key in places_table_data:
        places_table_data[key].append(np.sum(places_table_data[key]))

    additional_table = pd.DataFrame(data=np.array(list(places_table_data.values())), 
                                    index=places_table_data.keys(), 
                                    columns=[f'{col} ' for col in categories] + ['SUM'])
    return additional_table


def get_best_and_worst_rows(addded, categories):
    best = {}
    worst = {}
    for col in categories:
        if col == 'TO':
            best[col] = addded[col].min()
            worst[col] = addded[col].max()
            best[f'{col} '] = ''
            worst[f'{col} '] = ''
        elif col in ['Score', 'ExpScore']:
            scores_for_sort = []
            for sc in addded[col]:
                sc_values = list(map(float, sc.split('-')))
                scores_for_sort.append([sc_values[0], sc_values[2], sc_values[1]])
                max_val = max(scores_for_sort)
                best[col] = '-'.join(map(str, map(lambda x: x if x % 1.0 > 1e-7 else int(x), [max_val[0], max_val[2], max_val[1]])))
                min_val = min(scores_for_sort)
                worst[col] = '-'.join(map(str, map(lambda x: x if x % 1.0 > 1e-7 else int(x), [min_val[0], min_val[2], min_val[1]])))
        else:
            best[col] = addded[col].max()
            worst[col] = addded[col].min()
            if col not in ['TP', 'ER']:
                best[f'{col} '] = ''
                worst[f'{col} '] = ''
    best['SUM'] = ''
    worst['SUM'] = ''
    if 'ER' in categories:
        best['ER'] = ''
        worst['ER'] = ''
    return best, worst

In [5]:



def get_best_and_worst_rows_for_scores(table):
    best = {}
    worst = {}
    for col in table.columns:
        if col not in ['WD', 'LD', 'DD', 'W', 'D', 'L']:
            scores_for_sort = []
            for sc in table[col]:
                sc_values = list(map(float, sc.split('-')))
                scores_for_sort.append([sc_values[0], sc_values[2], sc_values[1]])
            max_val = max(scores_for_sort)
            best[col] = '-'.join(map(str, map(lambda x: x if x % 1.0 > 1e-7 else int(x), [max_val[0], max_val[2], max_val[1]])))
            min_val = min(scores_for_sort)
            worst[col] = '-'.join(map(str, map(lambda x: x if x % 1.0 > 1e-7 else int(x), [min_val[0], min_val[2], min_val[1]])))
        else:
            best[col] = ''
            worst[col] = ''
    return best, worst


def get_diff(expected, real):
    e = list(map(float, expected.split('-')))
    r = list(map(float, real.split('-')))
    return list(map(lambda x: np.round(x, 1), [e[0] - r[0], e[1] - r[1], e[2] - r[2]]))


def calc_team_win_stat(team_stat_list):
    team_res_np = np.array(team_stat_list)
    n_win = int(np.sum(team_res_np == 'W'))
    n_lose = int(np.sum(team_res_np == 'L'))
    n_draw = int(np.sum(team_res_np == 'D'))
    return '-'.join(map(str, [n_win, n_lose, n_draw]))


def get_matchup_result(team_stat, opp_stat, categories):
    win_sc = 0
    lose_sc = 0
    draw_sc = 0
    for index, cat in enumerate(categories):
        if team_stat[index] == opp_stat[index]:
            draw_sc += 1
        elif team_stat[index] > opp_stat[index]:
            if cat == 'TO':
                lose_sc += 1
            else:
                win_sc += 1
        else:
            if cat == 'TO':
                win_sc += 1
            else:
                lose_sc += 1
    if win_sc > lose_sc:
        return 'W'
    elif win_sc == lose_sc:
        return 'D'
    else:
        return 'L'

In [6]:
week = 8
leagues = [43767928, 134112, 99121987, 282844, 199973, 142634, 174837]
cat_subset = [43767928, 134112]
leagues = [43767928]
browser = webdriver.Chrome()
for league in leagues:
    link = f'https://fantasy.espn.com/basketball/league/scoreboard?leagueId={league}&matchupPeriodId={week}'
    browser.get(link)
    time.sleep(10)
    scoreboard_html_source = BeautifulSoup(browser.page_source)
    league_name = get_league_name(scoreboard_html_source)
    
    res, cat = get_week_results(scoreboard_html_source)
    scores_info = get_scores_info(res, cat)
    
    matchups_win_stats = {}
    for team in scores_info:
        week_wins = Counter()
        for opp in scores_info:
            if opp == team:
                continue
            fake_matchup_res = get_matchup_result(scores_info[team], scores_info[opp], cat)
            week_wins[fake_matchup_res] += 1
        tr = np.round((week_wins['W'] + week_wins['D'] * 0.5) / (len(scores_info) - 1), 2)
        matchups_win_stats[team] = tr if tr % 1.0 > 1e-7 else int(tr)
        
    vvv = pd.DataFrame(data=list(scores_info.values()), columns=cat, index=scores_info.keys())
    
    scores = get_week_stats(scoreboard_html_source, 'NBA')
    scores_dict = {}
    for sc in scores:
        scores_dict.update(sc)
        
    score_df = pd.DataFrame(data=list(scores_dict.values()), 
                            index=scores_dict.keys(), 
                            columns=['Score'])
    
    exp_score, exp_result = get_expected_score_and_result(res)
    exp_score_df =  pd.DataFrame(index=exp_score.keys(), columns=['ExpScore'], 
                                 data=list(exp_score.values()))
    matchups_df = pd.DataFrame(index=matchups_win_stats.keys(), columns=['TP'], 
                               data=list(matchups_win_stats.values()))
    week_res_df = pd.DataFrame(index=exp_result.keys(), columns=['ER'], 
                               data=list(exp_result.values()))
    add = get_places_table(vvv, cat)
    
    vvv = vvv.merge(score_df, how='outer', left_index=True, right_index=True)
    if league in cat_subset:
        vvv = vvv.merge(exp_score_df, how='outer', left_index=True, right_index=True)
    else:
        vvv = vvv.merge(matchups_df, how='outer', left_index=True, right_index=True)
        vvv = vvv.merge(week_res_df, how='outer', left_index=True, right_index=True)
    addded = vvv.merge(add, how='outer', left_index=True, right_index=True)
    addded = addded.sort_values(['SUM', 'PTS'])
    
    additional_cols = ['Score', 'TP', 'ER']
    if league in cat_subset:
        additional_cols = ['Score', 'ExpScore']
    to_add = pd.DataFrame(data=list(get_best_and_worst_rows(addded, cat + additional_cols)), 
                          index=['Best', 'Worst'])
    aaa = addded.append(to_add, sort=False)
    styles = [dict(selector='caption', props=[('text-align', 'center')])]
    aaa_styler = aaa.style.set_table_styles(styles).\
        apply(color_category_column, subset=pd.IndexSlice[aaa.index, set(additional_cols + cat) - {'ER'}]).\
        set_caption(f'{league_name}, week {week}').\
        apply(color_place_column, subset=pd.IndexSlice[addded.index, [f'{c} ' for c in cat]])
    if league not in cat_subset:
        aaa_styler = aaa_styler.applymap(color_matchup_result, subset=pd.IndexSlice[addded.index, ['ER']])
    display(aaa_styler)

,FG%,FT%,3PM,REB,AST,STL,BLK,TO,PTS,Score,ExpScore,FG%,FT%,3PM,REB,AST,STL,BLK,TO,PTS,SUM
Avtodor Saratov,0.4752,0.7841,58,277,159,43,46,79,714,6-3-0,6.9-2-0.1,3,4,5,1,2,2.5,1,7,2,27.5
Karma Bitches,0.4726,0.75,101,236,144,43,35,90,749,8-1-0,5.9-2.9-0.2,4,8.5,1,5,3,2.5,3,8,1,36
Burger Kings,0.4722,0.76,59,257,176,48,26,91,646,5-4-0,5.9-3-0.1,5,7,3.5,3,1,1,4,9,3,36.5
Minsk Shakals,0.4968,0.8045,25,241,115,31,44,59,641,4-5-0,5.7-3.3-0,2,2,9,4,6,5,2,4,5,39
Team Voevodskiy,0.513,0.7752,69,202,110,25,13,74,643,3-6-0,4.9-4-0.1,1,5,2,6,7,7,7.5,6,4,45.5
Salt-Lake Reapers,0.4465,0.8242,48,167,91,16,17,45,415,8-1-0,4.1-4.9-0,6,1,7,7,8,8,6,2,8,53
Team vidrilla,0.4431,0.7909,46,165,118,38,13,68,491,6-3-0,3.8-5.1-0.1,8,3,8,8,5,4,7.5,5,7,55.5
xxx xxx,0.4309,0.6552,59,276,124,30,22,93,584,1-8-0,3.8-5.1-0.1,9,10,3.5,2,4,6,5,10,6,55.5
Fantasy God,0.4464,0.7714,51,164,69,13,8,57,405,3-6-0,2.3-6.7-0,7,6,6,9,9,10,10,3,9,69
Tanking Experts,0.3508,0.75,19,84,48,15,11,23,192,1-8-0,1.3-7.6-0.1,10,8.5,10,10,10,9,9,1,10,77.5


In [7]:
cat_limit = 7
win_limit = 8
total_limit = 8
n_weeks = 8
leagues = [43767928, 134112, 99121987, 282844, 199973, 142634, 174837]
cat_susbet = [43767928, 134112]
leagues = [43767928]
browser = webdriver.Chrome()
for league in leagues:
    espn_scoreboard_url = 'https://fantasy.espn.com/basketball/league/scoreboard'
    links = [f'{espn_scoreboard_url}?leagueId={league}&matchupPeriodId={week+1}' for week in range(n_weeks)]
    
    all_scores = defaultdict(list)
    all_matchup_results = defaultdict(list)
    all_exp_scores = defaultdict(list)
    all_matchup_exp_results = defaultdict(list)
    all_matchups_won = defaultdict(list)
    
    for link in links:
        browser.get(link)
        time.sleep(10)
        scoreboard_html_source = BeautifulSoup(browser.page_source)
        league_name = get_league_name(scoreboard_html_source)

        res, cat = get_week_results(scoreboard_html_source)

        week_scores = get_scores_info(res, cat)
        for team in week_scores:
            week_wins = Counter()
            for opp in week_scores:
                if opp == team:
                    continue
                fake_matchup_res = get_matchup_result(week_scores[team], week_scores[opp], cat)
                week_wins[fake_matchup_res] += 1
            all_matchups_won[team].append('-'.join(map(str, [week_wins['W'], week_wins['L'], week_wins['D']])))
            
        scores = get_week_stats(scoreboard_html_source, 'NBA')
        scores_dict = {}
        
        opp_dict = {}
        for sc in scores:
            scores_dict.update(sc)
            opp_dict[sc[0][0]] = sc[1][0]
            opp_dict[sc[1][0]] = sc[0][0]
            
        for team in opp_dict:
            team_sc = np.array(scores_dict[team].split('-'))
            opp_sc = np.array(scores_dict[opp_dict[team]].split('-'))
            if compare_scores(team_sc, opp_sc):
                all_matchup_results[team].append('W')
            elif compare_scores(opp_sc, team_sc):
                all_matchup_results[team].append('L')
            else:
                all_matchup_results[team].append('D')
        
        for team in scores_dict:
            all_scores[team].append(scores_dict[team])

        exp_score, exp_result = get_expected_score_and_result(res)
        for team in exp_score:
            all_exp_scores[team].append(exp_score[team])
            all_matchup_exp_results[team].append(exp_result[team])

    for team in all_scores:
        stats = [np.array(list(map(float, score.split('-')))) for score in all_scores[team]]
        stats_array = np.vstack(stats)
        res = stats_array.sum(axis=0)
        all_scores[team].append('-'.join(list(map(lambda x: str(x) if x % 1.0 > 1e-7 else str(int(x)), res))))
        
    for team in all_exp_scores:
        stats = [np.array(list(map(float, score.split('-')))) for score in all_exp_scores[team]]
        stats_array = np.vstack(stats)
        res = list(map(lambda x: np.round(x, 1), stats_array.sum(axis=0)))
        all_exp_scores[team].append('-'.join(map(lambda x: str(x) if x % 1.0 > 1e-7 else str(int(x)), res)))
        
    for team in all_matchup_results:
        all_matchup_results[team].append(calc_team_win_stat(all_matchup_results[team]))
        
    for team in all_matchup_exp_results:
        all_matchup_exp_results[team].append(calc_team_win_stat(all_matchup_exp_results[team]))
    
    for team in all_matchups_won:
        stats = [np.array(list(map(float, score.split('-')))) for score in all_matchups_won[team]]
        stats_array = np.vstack(stats)
        res = list(map(lambda x: np.round(x, 1), stats_array.sum(axis=0)))
        all_matchups_won[team].append('-'.join(map(lambda x: str(x) if x % 1.0 > 1e-7 else str(int(x)), res)))
    
    table_data_dict = {team : all_exp_scores[team][len(all_exp_scores[team])-1-cat_limit:] for team in all_exp_scores}
    for team in table_data_dict:
        table_data_dict[team].append(all_scores[team][-1])
        table_data_dict[team].extend(get_diff(table_data_dict[team][-1], table_data_dict[team][-2]))
        
    table_win_data_dict = {team : all_matchup_exp_results[team][len(all_matchup_exp_results[team])-1-win_limit:] for team in all_matchup_exp_results}
    for team in table_win_data_dict:
        table_win_data_dict[team].append(all_matchup_results[team][-1])
        table_win_data_dict[team].extend(get_diff(table_win_data_dict[team][-1], table_win_data_dict[team][-2]))
    
    matchups_data_dict = {team: all_matchups_won[team][len(all_matchups_won[team]) - 1 - total_limit:] for team in all_matchups_won}
    for team in matchups_data_dict:
        matchups_data_dict[team].extend(map(int, matchups_data_dict[team].pop().split('-')))
    
    
    df_matchups = pd.DataFrame(data=list(matchups_data_dict.values()), 
                               index=matchups_data_dict.keys(), 
                               columns=[f'week {i+1}' for i in range(n_weeks-total_limit, n_weeks)] + ['W', 'L', 'D'])
    df_matchups = df_matchups.sort_values(['W', 'D'], ascending=False)
    to_add = pd.DataFrame(data=list(get_best_and_worst_rows_for_scores(df_matchups)), 
                          index=['Best', 'Worst'])
    df_matchups = df_matchups.append(to_add, sort=False)
    styles = [dict(selector='caption', props=[('text-align', 'center')])]
    df_matchups_styler = df_matchups.style.set_table_styles(styles).\
        set_caption(f'{league_name}').\
        apply(color_category_column, subset=[f'week {i+1}' for i in range(n_weeks-total_limit, n_weeks)])
    display(df_matchups_styler)
    
    if league not in cat_subset:
        df_win = pd.DataFrame(data=list(table_win_data_dict.values()), 
                              index=table_win_data_dict.keys(), 
                              columns=[f'week {i+1}' for i in range(n_weeks-win_limit, n_weeks)] + ['Total', 'ESPN', 'WD', 'LD', 'DD'])
        df_win = df_win.sort_values(['WD', 'DD'], ascending=False)
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        df_win_styler = df_win.style.set_table_styles(styles).\
            set_caption(f'{league_name}').\
            applymap(color_matchup_res, subset=[f'week {i+1}' for i in range(n_weeks-win_limit, n_weeks)]).\
            applymap(color_value, subset=pd.IndexSlice[table_data_dict.keys(), ['WD']])
        display(df_win_styler)
        
    if league in cat_subset:
        df = pd.DataFrame(data=list(table_data_dict.values()), 
                          index=table_data_dict.keys(), 
                          columns=[f'week {i+1}' for i in range(n_weeks-cat_limit, n_weeks)] + ['Total', 'ESPN', 'WD', 'LD', 'DD'])
        df = df.sort_values(['WD', 'DD'], ascending=False)
        to_add = pd.DataFrame(data=list(get_best_and_worst_rows_for_scores(df)), 
                              index=['Best', 'Worst'])
        df = df.append(to_add, sort=False)
        styles = [dict(selector='caption', props=[('text-align', 'center')])]
        df_styler = df.style.set_table_styles(styles).\
            set_caption(f'{league_name}').\
            apply(color_category_column, subset=pd.IndexSlice[df.index, [f'week {i+1}' for i in range(n_weeks-cat_limit, n_weeks)] + ['Total', 'ESPN']]).\
            applymap(color_value, subset=pd.IndexSlice[table_data_dict.keys(), ['WD']])
        display(df_styler)


,week 1,week 2,week 3,week 4,week 5,week 6,week 7,week 8,W,L,D
Avtodor Saratov,6-2-1,4-5-0,9-0-0,9-0-0,8-1-0,6-3-0,6-3-0,9-0-0,57,14,1
Burger Kings,3-6-0,9-0-0,6-3-0,4-5-0,8-1-0,9-0-0,8-1-0,7-2-0,54,18,0
xxx xxx,8-1-0,7-2-0,7-2-0,3-6-0,7-2-0,7-2-0,7-2-0,4-5-0,50,22,0
Karma Bitches,6-3-0,4-5-0,4-5-0,8-1-0,3-6-0,8-1-0,9-0-0,7-2-0,49,23,0
Team Voevodskiy,8-1-0,8-1-0,6-3-0,2-7-0,7-2-0,5-4-0,1-8-0,4-4-1,41,30,1
Minsk Shakals,1-8-0,5-4-0,1-8-0,6-3-0,5-4-0,2-6-1,5-4-0,7-2-0,32,39,1
Fantasy God,6-2-1,4-5-0,7-2-0,5-3-1,0-9-0,2-7-0,0-9-0,1-8-0,25,45,2
Tanking Experts,2-7-0,2-7-0,2-6-1,6-2-1,3-6-0,4-5-0,2-7-0,0-9-0,21,49,2
Salt-Lake Reapers,4-5-0,2-7-0,2-6-1,0-9-0,3-6-0,1-8-0,4-5-0,3-6-0,19,52,1
Team vidrilla,0-9-0,0-9-0,0-9-0,1-8-0,1-8-0,0-8-1,3-6-0,2-6-1,7,63,2


,week 2,week 3,week 4,week 5,week 6,week 7,week 8,Total,ESPN,WD,LD,DD
Karma Bitches,4.1-4.6-0.3,4.2-4.8-0,6-2.9-0.1,3.8-5.2-0,6.6-2.4-0,6.4-2.6-0,5.9-2.9-0.2,41.9-29.5-0.6,50-21-1,8.1,-8.5,0.4
Team Voevodskiy,5.9-3-0.1,5.1-3.8-0.1,2.3-6.4-0.2,5.6-3.4-0,4.6-4.3-0.1,2.1-6.8-0.1,4.9-4-0.1,36.5-34.6-0.8,41-30-1,4.5,-4.6,0.2
Team vidrilla,2.2-6.7-0.1,2-7-0,2.7-6-0.3,1.9-7-0.1,2.8-6-0.2,3.3-5.7-0,3.8-5.1-0.1,21.1-50.1-0.8,23-49-0,1.9,-1.1,-0.8
Fantasy God,4.7-4.2-0.1,5.2-3.7-0.1,5.3-3.4-0.2,2.1-6.8-0.1,3.4-5.4-0.1,2.2-6.7-0.1,2.3-6.7-0,29.9-41.1-0.8,31-39-2,1.1,-2.1,1.2
Salt-Lake Reapers,2.3-6.4-0.2,2.8-5.9-0.3,2.7-6.3-0,4.4-4.6-0,2.9-6.1-0,4.1-4.8-0.1,4.1-4.9-0,28-43.3-0.6,29-42-1,1,-1.3,0.4
Minsk Shakals,4.7-4.3-0,2.6-6.2-0.2,5.6-3.3-0.1,5-4-0,3.9-4.9-0.2,4.7-4.3-0,5.7-3.3-0,35.8-35.6-0.6,35-35-2,-0.8,-0.6,1.4
Tanking Experts,3.7-5.1-0.2,4-4.9-0.1,5.3-3.3-0.3,4.3-4.6-0.1,3.7-5.1-0.2,3.8-5.2-0,1.3-7.6-0.1,30-40.8-1.1,27-43-2,-3,2.2,0.9
Burger Kings,6.8-2.2-0,6-2.9-0.1,4.6-4.4-0,6-2.9-0.1,6.3-2.4-0.2,6.7-2.1-0.2,5.9-3-0.1,45.6-25.3-0.9,42-30-0,-3.6,4.7,-0.9
Avtodor Saratov,4.4-4.3-0.2,7.4-1.4-0.1,5.9-3-0.1,6.2-2.8-0,4.8-4-0.2,5.6-3.2-0.2,6.9-2-0.1,46.3-24.4-1.1,42-29-1,-4.3,4.6,-0.1
xxx xxx,5.6-3.4-0,5-3.8-0.2,3.8-4.9-0.3,5.4-3.6-0,5.4-3.6-0,5.7-3.2-0.1,3.8-5.1-0.1,40.7-30.6-0.7,34-36-2,-6.7,5.4,1.3
